# Binary Graph Classification
#### Done on the smallest class 457<br>using the vulnerable and safe file of this class only

In [ ]:
import pandas as pd
import numpy as np
import json
import torch
import os
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, Sequential, Linear, global_mean_pool
import pickle
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
import torch.nn.functional as F




from sklearn.model_selection import train_test_split

#from gcnHelpersGraph import GCN, preProcessingOneDataPoint, train, test

# Training on Binary class
#### please put the folders of the CVE's in the script's directory<br>then inside each folder put 2 subfolders, one for the nodes and the other for edges
##### also for the safe class, name the folder 457_SAFE, and inside it nodes_457_SAFE and edges_457_SAFE

In [ ]:

#List containing all input data of type Data (Pytorch geometric datastructure that holds a single graph)
allData = []

torch.cuda.empty_cache()

cve = '78'
print(f"-----------LOADING CVE {cve} Vulnerable------------------")


pathToEdges = f"D:/ClassWork/Guardista/4-Features_Extraction/{cve}/edges_{cve}"

with open (f'{cve}/features_matrices/features_matrices_'+cve+'.npy', 'rb') as f:
    features_matrices_list = np.load(f,  allow_pickle=True)

with open(f'{cve}/nodes_targets/nodes_targets_'+cve+'.npy', 'rb') as f:
    nodes_targets_list = np.load(f,  allow_pickle=True)


print(f"num Graphs : {features_matrices_list.shape[0]}")


# Reading the adjacency list of each graph
adj_Lists = []
for filename in os.listdir(pathToEdges):
    f = os.path.join(pathToEdges, filename)
    df = pd.read_csv(f, header=None)
    df.dropna(inplace=True)             # <---- here is the dropping
    adj_Lists.append(df)
    


numVulnNodes = 0


# Reading the label of each node
Targets_List = []
for target in nodes_targets_list:
    numVulnNodes += sum(target)
    Targets_List.append(1)
    

numNodes = 0
# Reading the features of each node
node_Features_List = []
for node_feature in features_matrices_list:
    n = np.array(node_feature,dtype='int64')
    numNodes+= n.shape[0]
    node_Features_List.append(torch.tensor(n, dtype=torch.float))


assert (len(adj_Lists) == len(Targets_List))
assert (len(adj_Lists) == len(node_Features_List))


# Convert the edge list to use 0-based indices
adj_Tensors = []
for adj in adj_Lists:
    # the documentation strictly says to convert the adjacency list to a contiguous list
    adj_Tensors.append(torch.as_tensor(adj.to_numpy(), dtype=torch.long).t().contiguous())   


#Iterate over each graph, make a Data object, then append to all our dataset
cntCorruptData=0
for i, adj in enumerate(adj_Tensors):
    d = Data(x=node_Features_List[i], edge_index=adj, y=torch.as_tensor([1]))  # <--- from the documentation.
    try:
        d.validate(raise_on_error=True)     # <--------- this line makes sure each input graph strictly follows the correct rules, to evade errors
    except:
        print('DIMENSION ERROR')
        print(f"We have features for {len(node_Features_List[i])} Nodes ")
        print(f"But the adjacency list contains {max(set(np.array(adj[0,:])))} Unique Nodes")
        cntCorruptData +=1
        continue
    allData.append(d)


print(f"number of corrupted files due to missing node features for certain nodes = {cntCorruptData}")
   

### Loading the 457 SAFE dataset

In [ ]:
print(f"-----------LOADING CVE {cve} SAFE------------------")


pathToEdges = f"D:/ClassWork/Guardista/4-Features_Extraction/{cve}_SAFE/edges_{cve}_SAFE"

with open (f'{cve}_SAFE/features_matrices/features_matrices_'+cve+'.npy', 'rb') as f:
    features_matrices_list = np.load(f,  allow_pickle=True)

with open(f'{cve}_SAFE/nodes_targets/nodes_targets_'+cve+'.npy', 'rb') as f:
    nodes_targets_list = np.load(f,  allow_pickle=True)


print(f"num Graphs : {features_matrices_list.shape[0]}")


# Reading the adjacency list of each graph
adj_Lists = []
for filename in os.listdir(pathToEdges):
    f = os.path.join(pathToEdges, filename)
    df = pd.read_csv(f, header=None)
    df.dropna(inplace=True)             # <---- here is the dropping
    adj_Lists.append(df)
    


numVulnNodes = 0


# Reading the label of each node
Targets_List = []
for target in nodes_targets_list:
    numVulnNodes += sum(target)
    Targets_List.append(0)
    

numNodes = 0
# Reading the features of each node
node_Features_List = []
for node_feature in features_matrices_list:
    n = np.array(node_feature,dtype='int64')
    numNodes+= n.shape[0]
    node_Features_List.append(torch.tensor(n, dtype=torch.float))


assert (len(adj_Lists) == len(Targets_List))
assert (len(adj_Lists) == len(node_Features_List))


# Convert the edge list to use 0-based indices
adj_Tensors = []
for adj in adj_Lists:
    # the documentation strictly says to convert the adjacency list to a contiguous list
    adj_Tensors.append(torch.as_tensor(adj.to_numpy(), dtype=torch.long).t().contiguous())   


#Iterate over each graph, make a Data object, then append to all our dataset
cntCorruptData=0
for i, adj in enumerate(adj_Tensors):
    d = Data(x=node_Features_List[i], edge_index=adj, y=torch.as_tensor([0]))  # <--- from the documentation.
    try:
        d.validate(raise_on_error=True)     # <--------- this line makes sure each input graph strictly follows the correct rules, to evade errors
    except:
        print('DIMENSION ERROR')
        print(f"We have features for {len(node_Features_List[i])} Nodes ")
        print(f"But the adjacency list contains {max(set(np.array(adj[0,:])))} Unique Nodes")
        cntCorruptData +=1
        continue
    allData.append(d)


print(f"number of corrupted files due to missing node features for certain nodes = {cntCorruptData}")

### Train-Test-Split and data loader

In [ ]:

# Train Test Split
allData_train, allData_test = train_test_split(allData, test_size=0.3, shuffle=True)
allData_train, allData_val = train_test_split(allData_train, test_size=0.1, shuffle=True)


unique_classes = set()

for d in allData_train:
    unique_classes.add(int(d.y.numpy()))

print(f"Unique classes {unique_classes}")


#Our Data Loader
batch_size = 5
loader = DataLoader(allData_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(allData_val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(allData_test, batch_size=batch_size, shuffle=True)

# GCN Class

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

numClasses = 2





class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, numClasses, numFeatures):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        
        self.conv1 = GCNConv(numFeatures, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.conv5 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, numClasses)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        # x = self.gat1(x, edge_index)
        # x = x.relu()
        # x = self.gat2(x, edge_index)
        # x = x.relu()
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        
        x = self.conv4(x, edge_index)
        x = x.relu()
        x = self.conv5(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        Embeddings = x

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x, torch.squeeze(Embeddings)








def train(model, loader, device, optimizer, criterion):
    model.train()

    for data in loader:  # Iterate in batches over the training dataset.
         out,_ = model(data.x.to(device=device), data.edge_index.to(device=device), data.batch.to(device=device))  # Perform a single forward pass.
         loss = criterion(out, data.y.to(device=device))  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.
         torch.cuda.empty_cache()



def test(model, loader, device):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out,_ = model(data.x.to(device=device), data.edge_index.to(device=device), data.batch.to(device=device))  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y.to(device=device)).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


def inference(model, data_point, device):
    model.eval().to(device=device)
    with torch.no_grad():
        x = data_point.x.to(device=device)
        edge_index = data_point.edge_index.to(device=device)
        batch = data_point.batch
        out, embedding = model(x, edge_index, batch)
        pred = out.argmax(dim=1).cpu().item()
    return pred, embedding






model = GCN(hidden_channels=256, numClasses=numClasses, numFeatures=33).to(device=device)
print(model)



optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-6)
#penaltyWeights = torch.tensor( Class_Weights)  # <--- THOSE WEIGHTS ARE COMPUTED MANUALLY ACCORDING TO AN EQUATION
criterion = torch.nn.CrossEntropyLoss()



for epoch in range(1, 3):
    train(model, loader, device, optimizer, criterion)
    train_acc = test(model, loader, device)
    val_acc = test(model, val_loader, device)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Validation Acc: {val_acc:.4f}')





# Saving the model
with open(f'GCN_{cve}_binary.pkl', 'wb') as f:
    pickle.dump(model.cpu(), f)






# Testing

In [ ]:
from sklearn.metrics import classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device=device)

y_true = []
y_predicted = []
unique_classes = set()
for d in allData_test:
    y_true.append(d.y.cpu().numpy())
    
    pred,_ = inference(model, d.to(device=device), device=device)
    y_predicted.append(pred)


print(classification_report(y_true= y_true, y_pred= y_predicted))

In [ ]:
import gcnHelpers
pathToUser_Edges = "sys.argv[1]"
outputPath = "sys.argv[2]"
cve = "sys.argv[3]"
modelPath = "sys.argv[4]"
npyPath = outputPath+'/features_matrices/features_matrices_' + cve + '.npy'
classification, df = gcnHelpers.InferenceGCN(modelPath, pathToUser_Edges, outputPath, 'true', cve, npyPath)

print (df.head())
        

In [ ]:
print (df.head())